In [10]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [23]:
train = pd.read_csv('../data/train_featureV1.csv')
test = pd.read_csv('../data/test_featureV1.csv')

In [24]:
train.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_5_x,voice_opp_len_6_x,voice_opp_len_7_x,voice_opp_len_8_x,voice_opp_len_9_x,...,visit_dura_x_y.10,visit_dura_y_y.10,visit_dura_x_y.11,visit_dura_y_y.11,visit_dura_x_y.12,visit_dura_y_y.12,visit_dura_x_y.13,visit_dura_y_y.13,visit_dura_x_y.14,visit_dura_y_y.14
0,u0001,0,22.0,79.0,17.0,1.0,0.0,0.0,0.0,0.0,...,47692.0,13190.0,17314.0,14337.0,34536.0,17084.0,4806.0,9402.0,3378084.0,8907.0
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,u0003,0,15.0,21.0,10.0,1.0,0.0,0.0,2.0,0.0,...,25104.0,17164.0,18219.0,38717.0,1039108.0,440294.0,175357.0,2184.0,452927.0,2281.0
3,u0004,0,77.0,254.0,31.0,0.0,0.0,0.0,3.0,0.0,...,145443.0,22237.0,10694.0,513625.0,110245.0,440247.0,607104.0,15360.0,1971461.0,5034.0
4,u0005,0,55.0,401.0,28.0,1.0,0.0,0.0,0.0,0.0,...,2343340.0,0.0,0.0,4844.0,82539.0,9950.0,0.0,1958.0,174052.0,160872.0


In [29]:
xgb_params = {
    'booster':'gbtree',
    'objective':'binary:logistic',
    'stratified':True,
    'max_depth':30,
    # 'gamma':1,
    'subsample':0.3,
    #'colsample_bytree':0.8,
    # 'lambda':1,
    'eta':0.01,
    'seed':20,
    'silent':1,
}


def evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    pre.preds=pre.preds.map(lambda x: 1 if x>=0.38 else 0)
    f1 = metrics.f1_score(pre.label,pre.preds)
    res = 0.6*auc +0.4*f1
    return 'res',res

### 本地CV

In [30]:
dtrain = xgb.DMatrix(train.drop(['uid','label'],axis=1),label=train.label)
xgb.cv(xgb_params,dtrain,num_boost_round=500,nfold=3,verbose_eval=10,early_stopping_rounds=100,maximize=True,feval=evalMetric)

[0]	train-error:0.116824+0.00633592	train-res:0.662653+0.0112148	test-error:0.155029+0.00635095	test-res:0.636051+0.00471648
[10]	train-error:0.0712147+0.00368613	train-res:0.704691+0.00170316	test-error:0.109823+0.00977893	test-res:0.678663+0.0103499
[20]	train-error:0.069214+0.000507246	train-res:0.707363+0.00173994	test-error:0.109623+0.0108094	test-res:0.680489+0.010624
[30]	train-error:0.0639127+0.000498085	train-res:0.783794+0.0117786	test-error:0.107423+0.00893018	test-res:0.744214+0.00376663
[40]	train-error:0.0627123+0.000742617	train-res:0.871432+0.00758302	test-error:0.108823+0.00879831	test-res:0.821066+0.00795881
[50]	train-error:0.0621123+0.00149509	train-res:0.896531+0.00232804	test-error:0.106822+0.00850869	test-res:0.841598+0.0130673
[60]	train-error:0.060012+0.000432749	train-res:0.908064+0.00216997	test-error:0.105022+0.00946256	test-res:0.851913+0.0165019
[70]	train-error:0.0562117+0.000515491	train-res:0.914314+0.00169243	test-error:0.104822+0.00964257	test-res:0.8

,test-error-mean,test-error-std,test-res-mean,test-res-std,train-error-mean,train-error-std,train-res-mean,train-res-std
0,0.155029,0.006351,0.636051,0.004716,0.116824,0.006336,0.662653,0.011215
1,0.133429,0.009378,0.653969,0.012046,0.096620,0.002423,0.681115,0.004352
2,0.122825,0.008676,0.664561,0.009395,0.088118,0.002467,0.690214,0.003373
3,0.119024,0.008456,0.671411,0.007244,0.082117,0.004821,0.696196,0.003458
4,0.119224,0.007633,0.674385,0.008455,0.080916,0.004966,0.697795,0.003365
5,0.114624,0.007790,0.674661,0.009457,0.078316,0.003018,0.699766,0.003099
6,0.111624,0.009780,0.676770,0.010035,0.077016,0.005039,0.701639,0.002966
7,0.112624,0.010580,0.676646,0.009573,0.077116,0.003323,0.702377,0.003531
8,0.109623,0.009496,0.676711,0.009976,0.075715,0.003049,0.703324,0.002418
9,0.110023,0.008801,0.677388,0.010425,0.073615,0.004321,0.703821,0.002391


## 训练

In [6]:
model=xgb.train(xgb_params,dtrain=dtrain,num_boost_round=500,verbose_eval=10,
                evals=[(dtrain,'train')],maximize=True,feval=evalMetric,early_stopping_rounds=100)

[0]	train-error:0.15223	train-res:0.585907
Multiple eval metrics have been passed: 'train-res' will be used for early stopping.

Will train until train-res hasn't improved in 100 rounds.
[10]	train-error:0.142629	train-res:0.599864


KeyboardInterrupt: 

### 预测

In [ ]:
#pred=model.predict(test.drop(['uid'],axis=1))
dtest = xgb.DMatrix(test.drop(['uid'],axis=1))
preds =model.predict(dtest)

In [130]:
res =pd.DataFrame({'uid':test.uid,'label':preds})
res.head()

,label,uid
0,0.596163,u7000
1,0.723701,u7001
2,0.016688,u7002
3,0.015570,u7003
4,0.129746,u7004


In [132]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.38 else 0)
#res.label = res.label.map(lambda x: int(x))
res.head()

,label,uid
1424,1,u8424
907,1,u7907
439,1,u7439
609,1,u7609
1751,1,u8751


In [133]:
res.to_csv('../result/lgb-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])